This notebook uses the prebuilt `executorch` pip package to export and run an XNNPACK-delegated model. This shows that it's possible to do so without needing to clone the repo and install from source, simplifying the steps currently at https://pytorch.org/executorch/stable/getting-started-setup.html.

Note that this will only work if the model is compatible with the operators and backends linked into the `executorch>=0.2.0` package: i.e., it uses the core ATen operator set, and may use the XNNPACK backend. But this example doesn't support custom operators or other backends.

Install the prebuilt executorch pip package.

NOTE: You may see the message ERROR: pip's dependency resolver ... for packages like torchaudio and torchtext, but it won't affect this demo.

In [ ]:
!pip install executorch==0.2.1

# Instead of 0.2.1, use the latest version in https://pypi.org/project/executorch/#history

Demonstrate that the packages are imported successfully.

In [ ]:
from executorch import version

version.__version__

In [ ]:
import torch

torch.__version__

# executorch pip package uses torch as dependency

Demonstrate that the native pybindings module imports successfully, and provides basic operators.

Pybindings are great for doing testing and prototyping. Ultimately, you'll need to write the executorch runtime integration in C++ during productionization.

In [ ]:
from executorch.extension.pybindings import portable_lib

ops = portable_lib._get_operator_names()
f"Found {len(ops)} ops; first is '{ops[0]}'"

Let's do inference on MobileNetV2.

In order to do that,
- first we need to find the nn.Module
- use `torch.export()` to export the graph
- And subsequently, generate `.pte` file
- Lastly, do inference via executorch runtime on the `.pte` file

In [ ]:
!pip install 'torchvision==0.18.1'

In [ ]:
from torchvision.models import mobilenet_v2
from torchvision.models.mobilenetv2 import MobileNet_V2_Weights

mv2 = mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT) # This is torch.nn.Module

In [ ]:
from executorch.exir import to_edge
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner

model = mv2.eval() # turn into evaluation mode

example_inputs = (torch.randn((1, 3, 224, 224)),) # Necessary for exporting the model

exported_graph = torch.export.export(model, example_inputs) # Core Aten graph

edge = to_edge(exported_graph) # Edge Dialect

edge_delegated = edge.to_backend(XnnpackPartitioner()) # Parts of the graph are delegated to XNNPACK

executorch_program = edge_delegated.to_executorch() # ExecuTorch program

In [ ]:
pte_path = "mv2_xnnpack.pte"

with open(pte_path, "wb") as file:
    executorch_program.write_to_file(file) # Serializing into .pte file

Try loading and executing the XNNPACK-delegated model using the executorch pip package.

In [ ]:
from executorch.extension.pybindings import portable_lib
m = portable_lib._load_for_executorch(pte_path)

t = torch.randn((1, 3, 224, 224))

output = m.forward([t])
assert len(output) == 1, f"Unexpected output length {len(output)}"
assert output[0].size() == torch.Size([1, 1000]), f"Unexpected output size {output[0].size()}"
print("PASS")